In [ ]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
def normalize_landmarks(landmarks):
    x_values = []
    y_values = []
    for landmark in landmarks:
    #if landmark.visibility >= 0.50:
        x_values.append(landmark.x)
        y_values.append(landmark.y)
    
    min_x = min(x_values)
    min_y = min(y_values)
    normalized_x  = [x - min_x for x in x_values]

    normalized_y = [y- min_y for y in y_values]
    #normalized_landmarks = list(zip(normalized_x, normalized_y))
    normalized_landmarks = [item for pair in zip(normalized_x, normalized_y) for item in pair]
    #print(normalized_landmarks)
    return normalized_landmarks
    


In [ ]:
def isnarroworwide(feetlandmarks):
    #feetlandmarks  = feetlandmarks[-4:]
    legonedifference = feetlandmarks[1] - feetlandmarks[3]
    legtwodifference = feetlandmarks[2] - feetlandmarks[0]
    #print(legonedifference, ' ', legtwodifference)
    if (legonedifference > -0.01 and legonedifference < 0.01) or (legtwodifference > -0.01 and legtwodifference < 0.01) :
        status = 'perfect'
    elif legonedifference > 0.01 or legtwodifference > 0.01:
        status = 'wide'
    elif legonedifference < -0.01 or legtwodifference < -0.01:
        status = 'narrow'

    else:
        status = 'NA'
    return status
    

In [ ]:
def are_legs_too_open_or_closed(landmarks):
    diff1 = landmarks[30] - landmarks[12]
    diff2 = landmarks[11] -  landmarks[29]
    if (diff1 > -0.015 and diff1 < 0.015) or (diff2 > -0.015 and diff2 < 0.015):
        status = 'perfect'
    elif diff1< -0.015 or diff2< -0.015:
        status = 'too open'
    elif diff1 > 0.015 or diff2< 0.015:
        status = 'too closed'
    else:
        status = 'NA'
    
    return status

In [ ]:
def calculateangle(a,b,c): 
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians*180/np.pi)
    #print(angle)
    if angle > 180:
        angle = 360 - angle
    return angle

In [ ]:
class checkExcercise:

    def __init__(self, state):
        self.state = state

    def calculateangle(self,a,b,c): 
        a = np.array(a)
        b = np.array(b)
        c = np.array(c)

        radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
        angle = np.abs(radians*180/np.pi)
        #print(angle)
        if angle > 180:
            angle = 360 - angle
        return angle
    

    def isBicepCurl(self,results,mode='left'):
        #print('mode is ', mode)
        if mode == 'left':
            #print(mode)
            landmarks = results.pose_landmarks.landmark
            lshoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            lelbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            lwrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            angle = self.calculateangle(lshoulder,lelbow,lwrist)
            
            if angle > 160:
                #print('down')
                self.state = 'down'
                #print('gottohere')
            if angle < 30 and self.state =='down':
                #print('up')
                self.state = 'up'
                #print('gottohere')
                return True
                #print(counter)
        elif mode == 'right':
            landmarks = results.pose_landmarks.landmark
            rshoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            relbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            rwrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            angle = self.calculateangle(rshoulder,relbow,rwrist)
            if angle > 160:
                #print('down')
                self.state = 'down'
            if angle < 30 and self.state =='down':
                #print('up')
                self.state = 'up'
                return True
        elif mode == 'both':
            landmarks = results.pose_landmarks.landmark
            rshoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            relbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            rwrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            lshoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            lelbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            lwrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            rangle = self.calculateangle(rshoulder,relbow,rwrist)
            langle = self.calculateangle(lshoulder,lelbow,lwrist)
            if rangle > 160 and langle > 160:
                #print('down')
                self.state = 'down'
            if rangle < 30 and langle < 30 and self.state =='down':
                #print('up')
                self.state = 'up'
                return True
            

In [ ]:
model = keras.models.load_model('model2.h5')
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(['up', 'down'])
counter = 0 
state = None
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5
font_thickness = 1
font_color = (255, 255, 255)
cap = cv2.VideoCapture(0)
mode = input('which mode? squats or bicepcurls')
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    if mode == 'squats':
        while cap.isOpened():
            ret, frame = cap.read()

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            #image = cv2.resize(image, 480,640)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            try:
                landmarks = results.pose_landmarks.landmark
                
                normalized_landmarks = normalize_landmarks(landmarks)
                testdata = np.array([normalized_landmarks])
                predictions = model.predict(testdata)
                predicted_classes = (predictions > 0.5).astype(int)

                #predicted_classes = np.argmax(predictions, axis=1)
                upordown = label_encoder.inverse_transform(predicted_classes.flatten()).tolist()[0]
                landmarks = [landmark.x for landmark in landmarks]
                #print(f'Predicted Class: {predicted_classes_labels}')  
                #normalized_landmarks = normalize_landmarks(landmarks)
                #print(isnarroworwide(landmarks[-4:]))
                narroworwide = isnarroworwide(landmarks[-4:])
                too_open_or_closed = are_legs_too_open_or_closed(landmarks)

                if narroworwide == 'perfect' and too_open_or_closed == 'perfect':
                     if upordown == 'down':
                          state == 'down'
                     if upordown == 'up' and state =='down':
                          state == 'up'
                          counter += 1
                y_position = 30
                text_lines = [f'Toes status: {narroworwide}', f'Legs width status: {too_open_or_closed}', f'Number of reps: {counter}', f'Position: {upordown}']
                for line in text_lines:
                    cv2.putText(image, line, (10, y_position), font, font_scale, font_color, font_thickness, cv2.LINE_AA)
                    y_position += 30
                # diff1 = landmarks[30] - landmarks[12]
                # diff2 = landmarks[11] -  landmarks[29] 
                #print(diff1, 'second: ', diff2)
                # cv2.putText(image, f'''second:{diff2}''',(50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                #cv2.putText(image, f'norw: {narroworwide}, oc: {too_open_or_closed}, pos:{predicted_classes_labels}',(50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                #print(image.shape)
                #print(topandbottom(landmarks))
                #topandbottom(landmarks,resolution)
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                        ) 
            except Exception as e:
                print(e)
                #pass
            
            #print(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value])
            
            cv2.imshow('Mediapipe Feed', image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                    break


    elif mode == 'bicep':

             counter = 0 

             checkExcercise = checkExcercise(state='None')
             excerciseMode = input('which bicepcurl mode?')

             while cap.isOpened():
                ret, frame = cap.read()
                
                # Recolor image to RGB
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False
            
                # Make detection
                results = pose.process(image)
            
                # Recolor back to BGR
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                
                # Extract landmarks
                try:
                    isbicepcurl = checkExcercise.isBicepCurl(results,mode=excerciseMode)
                    #print('yay')
                    #cv2.putText(image, str(angle), 
                    #               tuple(np.multiply(lelbow, [640, 480]).astype(int)), 
                    #               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                    #                    )

                    if isbicepcurl:
                        counter += 1
                        # print(counter)
                    y_position = 30
                    #print(counter)
                    cv2.putText(image, str(counter), (10, y_position), font, font_scale, font_color, font_thickness, cv2.LINE_AA)
                except Exception as e:
                    #pass
                    print(e)
                
                
                # Render detections
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                        )               
                
                cv2.imshow('Mediapipe Feed', image)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

    


    print('releasing')
    cap.release()
    cv2.destroyAllWindows()
